In [1]:
import os
import torch
import mlflow_env
from gpt import GPTLanguageModel, GPT2Extended
from gpt_params import tokenizer

/Users/joaquinpolonuer/Documents/venvs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)

experiment = "Transformer"
run_id = "1631cdf63904427fb5833afa9372b625"

gpt_model = GPTLanguageModel.load_from_mlflow(experiment, run_id, device)
gpt_2_model = GPT2Extended.from_pretrained("gpt2").to(device)

/Users/joaquinpolonuer/Documents/software/NLP-monosemanticity/gpt.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=dev

In [6]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("He was"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

He was hypothesized : only in portions of Frieduidi & Pradz song " The American� <unk> " ) ; the American Library Journal received two license to breed a two @-@ consecutive subspecies : One Moreks , in which they helped Adult Swim , University of Governments , notablyet : Max Borke Nima Huyck , bronchophon , <unk> Jarvis , Rd Ayandero , Montyče , II , Shadow Tree , BukainOn Lafo published a


In [4]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("The quick brown fox jumps over the lazy dog"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.greedy_generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

The quick brown fox jumps over the lazy dog . The female is a white @-@ billed , and the female is a white @-@ white , and white , with a white white white white , and white , with a white white white white white , and white , with a white white white white white , and white , with a white white white white white , and white , with a white white white white white , and white , with a white white white white white , and white , with a white white white white white , and white


In [5]:
sample_output = gpt_2_model.generate(
    idx,
    max_new_tokens=40,
    do_sample=False,
)
print(tokenizer.decode(sample_output.squeeze().tolist()))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The quick brown fox jumps over the lazy dog and runs off.

"I'm sorry, I'm sorry, I'm sorry," the fox says.

"I'm sorry, I'm sorry, I'm sorry," the fox
